In [7]:
import torch
from torch import nn
from torch.nn import functional as F

X = torch.rand(2, 20)

In [8]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))
    
net = MLP()
net(X)

tensor([[-0.1608, -0.1915,  0.1024, -0.0219,  0.1882, -0.1084, -0.0680,  0.0038,
          0.0502, -0.0754],
        [-0.0050, -0.1162,  0.0870,  0.0445,  0.1086, -0.1115, -0.0100,  0.1135,
          0.0670, -0.1039]], grad_fn=<AddmmBackward0>)

In [9]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X
    

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0739,  0.0483, -0.0110,  0.0781, -0.0605, -0.3477,  0.0485, -0.2587,
         -0.0772,  0.1738],
        [-0.0582,  0.0188, -0.0352,  0.0253, -0.0982, -0.3114,  0.0183, -0.2263,
         -0.0762,  0.1218]], grad_fn=<AddmmBackward0>)

In [12]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    
net = FixedHiddenMLP()
net(X)

tensor(-0.3884, grad_fn=<SumBackward0>)

In [13]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
        nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    def forward(self, X):
        return self.linear(self.net(X))
    
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)


tensor(-0.0905, grad_fn=<SumBackward0>)